## Setup

To run this notebook you will need:
- a GPU
- To have installed the corresponding versions of CUDA Toolkit and cuDNN library to your GPU.

1. After that you need to create a new environment in anaconda as follows:
```
    conda create -n tf_gpu python=3.8 
```
2. Install the corresponding packages via the anaconda prompt:
```
    conda activate tf_gpu
    conda install -c anaconda tensorflow-gpu keras-gpu
    conda install numpy=1.18.5
    conda install cudatoolkit=10.1
```
3. Configure the Jupyter kernel
```
python -m ipykernel install --user --name tf_gpu --display-name "Python (GPU)"
```
4. Install packages
``` python
pip install Pillow
pip install tensorflow==2.3.0

#### Packages

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorboard.plugins.hparams import api as hp
import numpy as np
import scipy.io
import os
from PIL import Image

# %load_ext tensorboard


### Assesing setup

In [6]:
# MUST print TRUE
tf.test.is_built_with_cuda()

True

In [7]:
# MUST LIST YOU GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 665669926521770458
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12598196140159282792
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3028759348
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17604719507136690646
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15578366525513547133
physical_device_desc: "device: XLA_GPU device"
]


In [6]:
# THERE IS A BUG HERE WHERE YOU CANNOT TURN OFF THE LOG DEVICE PLACEMENT, SO UNCOMMENT THIS LINES CHECK THAT IT IS USING YOU GPU< THEN RESTART THE KERNEL AND COMMENT THESE LINES
# MUST USE YOUR GPU FOR MATRIX MULTIPLICATION
# tf.debugging.set_log_device_placement(True)
a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
# 
# c = tf.matmul(a, b)
# 
# print(c)

In [19]:
tf.debugging.set_log_device_placement(False)

In [7]:
c = tf.matmul(a, b)

#### Dataset

Here we will be using the same dataset as the previous assignments in which the problem is to classify a set of images as cats or not retrieved from https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification?select=train. We removed the picture "dog_505.png" as it caused problems while preprocessing it. We will first process only 100 images per class (100 cats and 100 dogs for training and another 100 cats and 100 dogs for validation) to reduce their pixel resolution to the same as the other assignments (64 pixels x 64 pixels) and then represent them as arrays.

In [9]:
def preprocess_dataset(dataset):
    dataset_flatten = dataset.reshape(dataset.shape[0],-1).T
    return dataset_flatten/255 

def process_images(directory, target_size=(64, 64), image_range=range(0,100)):
    image_list = []

    filenames = sorted(os.listdir(directory))[min(image_range):max(image_range)+1]
    
    for filename in filenames:
        if filename.endswith(".jpg"):
            file_path = os.path.join(directory, filename)
            
            img = Image.open(file_path)
            img_resized = img.resize(target_size)
            img_array = np.array(img_resized)
            
            image_list.append(img_array)
    
    return np.array(image_list)

def join_cats_and_dogs(cat_images,dog_images):
    cat_set_X = preprocess_dataset(cat_images)
    dog_set_X = preprocess_dataset(dog_images)
    m_cat_set = cat_set_X.shape[1]
    cat_set_Y = np.ones((1, m_cat_set))
    m_dog_set = dog_set_X.shape[1]
    dog_set_Y = np.zeros((1, m_dog_set))

    set_X = np.concatenate((cat_set_X, dog_set_X), axis=1)
    set_Y = np.concatenate((cat_set_Y, dog_set_Y), axis=1)

    np.random.seed(1)
    shuffle_indices = np.random.permutation(set_X.shape[1])
    final_set_X = set_X[:, shuffle_indices]
    final_set_Y = set_Y[:, shuffle_indices]

    return final_set_X,final_set_Y


In [10]:
train_cat_images = process_images("datasets/train/cats")
train_dog_images = process_images("datasets/train/dogs")

test_cat_images = process_images("datasets/train/cats",image_range=(101,200))
test_dog_images = process_images("datasets/train/dogs",image_range=(101,200))

x_train, y_train = join_cats_and_dogs(train_cat_images,train_dog_images)
y_train = y_train.flatten()
x_train = x_train.T


x_test, y_test = join_cats_and_dogs(test_cat_images,test_dog_images)
y_test = y_test.flatten()
x_test = x_test.T


## Tuning Parameters

In [56]:
import os
import shutil
log_dir = r'logs\hparam_tuning'
shutil.rmtree(log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [57]:
os.path.exists(log_dir)

True

### Grid Search
We will tune 4 parameters: the number of units, the dropout frequency, the batch size and the optimizer used. 

In [52]:
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.05, 0.10, 0.15, 0.20]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([16, 32]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

In [53]:
# model = tf.keras.models.Sequential([
#     # Add a Reshape layer to unflatten the input
#     tf.keras.layers.Reshape((64, 64, 3), input_shape=(12288,)),
#     
#     # Block 1
#     tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
#     
#     # Block 2
#     tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
#     
#     # Flatten the output of the conv layers to feed into the dense layers
#     tf.keras.layers.Flatten(),
#     
#     # Fully connected layers with dropout in between
#     tf.keras.layers.Dense(4096, activation='relu'),
#     tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
#     tf.keras.layers.Dense(4096, activation='relu'),
#     tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
#     
#     # Output layer
#     tf.keras.layers.Dense(2, activation='softmax'),  # Adjust the number of units for the number of classes
# ])

In [54]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
        # Reshape layer to unflatten the input
        tf.keras.layers.Reshape((64, 64, 3), input_shape=(12288,)),
        
        # Block 1 - Simplified to one Conv2D layer
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Block 2 - Removed entirely or you can add another with fewer filters if necessary
        # tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        # tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        # Flatten the output of the conv layers to feed into the dense layers
        tf.keras.layers.Flatten(),
        
        # Fully connected layers with dropout in between and reduced size
        tf.keras.layers.Dense(1024, activation='relu'),  # Reduced from 4096 to 1024
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        
        # Output layer - adjust the number of units for the number of classes
        tf.keras.layers.Dense(2, activation='softmax'),
    ])

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  callbacks = [
  keras.callbacks.TensorBoard(
  log_dir=log_dir,
  histogram_freq=1,
  embeddings_freq=1,
  )
  ]

  model.fit(x_train, y_train, epochs=1,callbacks=callbacks, batch_size=hparams[HP_BATCH_SIZE]) 
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def run(run_dir, hparams):
    full_run_dir = os.path.join('logs', 'hparam_tuning', run_dir)
    ensure_dir(full_run_dir)  # Ensure the directory exists
    with tf.summary.create_file_writer(full_run_dir).as_default():
        hp.hparams(hparams)  # Record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [63]:
log_dir

'logs\\hparam_tuning'

In [67]:
session_num = 0

for dropout_rate in HP_DROPOUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for batch_size in HP_BATCH_SIZE.domain.values:  # Add this loop
            hparams = {
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
                HP_BATCH_SIZE: batch_size,  # Include batch size
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run(log_dir + run_name, hparams)
            session_num += 1


--- Starting trial: run-0
{'dropout': 0.05, 'optimizer': 'adam', 'batch_size': 16}
12/13 [==========================>...] - ETA: 0s - loss: 10.2453 - accuracy: 0.4688

NotFoundError: Failed to create a NewWriteableFile: logs\hparam_tuning\train\keras_embedding.ckpt-0_temp_631780fe02b643f1b935e85f9a6fdc6a/part-00000-of-00001.data-00000-of-00001.tempstate737473315795254130 : The system cannot find the path specified.
; No such process [Op:SaveV2]

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10265223786447933628
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13037884004019447277
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10747635510916229690
physical_device_desc: "device: XLA_GPU device"
]


In [ ]:
tf.test.is_built_with_cuda()

True

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.config.list_physical_devices('GPU')

from tensorflow.python.client import device_lib

device_lib.list_local_devices()


Num GPUs Available:  0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2355745228207664348,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11784309633994895745
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8026317427330174693
 physical_device_desc: "device: XLA_GPU device"]

In [ ]:
tf.debugging.set_log_device_placement(True)
a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.constant([[1.0, 1.0], [0.0, 1.0]])

# Run a matrix multiplication operation and log the device that it's executed on.
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Using GPU for computation
tf.Tensor(
[[1. 3.]
 [3. 7.]], shape=(2, 2), dtype=float32)


In [ ]:
tf.config.list_physical_devices("GPU")

[]

In [ ]:
tf.config.list_physical_devices('GPU')

[]

In [ ]:
%tensorboard --logdir=logs/hparam_tuning

Reusing TensorBoard on port 6016 (pid 29612), started 0:00:14 ago. (Use '!kill 29612' to kill it.)